In [1]:
!pip install -U "transformers>=4.51.0" "accelerate>=0.34.0" "safetensors"
!pip install fastapi uvicorn pyngrok nest_asyncio

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ==========================================
# 모델 로드 (Chanwoo5954 파인튜닝 모델)
# ==========================================
MODEL_ID = "Chanwoo5954/qwen3-4b-defamation-top8"

print("📥 파인튜닝된 Qwen3-4B 모델 로딩 중...")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True
)

# 모델 로드 (GPU 직결)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,   # GPU 사용 시 float16 권장
    device_map="cuda",
    trust_remote_code=True
)

model.eval()
print("✅ 모델 로딩 완료! (Chanwoo5954/qwen3-4b-defamation-top8)")

📥 파인튜닝된 Qwen3-4B 모델 로딩 중...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 모델 로딩 완료! (Chanwoo5954/qwen3-4b-defamation-top8)


In [ ]:
import threading
import uvicorn
import time
import json
from fastapi import FastAPI, Request
from pyngrok import ngrok
import nest_asyncio

# ==========================================
# FastAPI 앱 및 예측 로직 설정
# ==========================================
app = FastAPI()

SYSTEM_PROMPT = """
너는 한국 형사 명예훼손 판결 데이터를 학습한 법률 특화 모델이다.
입력으로는 사건의 사실관계 요약(사실관계)만 주어진다.
너는 이 사실관계를 바탕으로 다음 항목을 JSON 형식으로 예측해야 한다.
반드시 아래 키만 포함된 하나의 JSON 객체만 출력하라.

{
  "판단": "유죄" | "무죄" | "일부유죄" | "기타",
  "형량": {
    "형종": "징역" | "벌금" | "무죄" | "기타",
    "형기_월": 정수 또는 null,
    "벌금액": 정수 또는 null,
    "집행유예_여부": true 또는 false 또는 null,
    "집행유예_기간_월": 정수 또는 null,
    "사회봉사_시간": 정수 또는 null,
    "추가_조건": 문자열 (없으면 빈 문자열 "")
  },
  "양형이유": "판결문 양형 이유를 요약한 한국어 문장 (없으면 빈 문자열)",
  "casenames": ["사건 유형(예: '명예훼손')"],
  "죄명": ["정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)" 등]
}

출력 형식 주의:
- 반드시 JSON 객체 하나만 출력하고, 그 밖의 설명, 자연어 문장은 절대 출력하지 마라.
- JSON 앞뒤에 ```json, ``` 같은 마크다운 코드를 붙이지 마라.
"""

@app.post("/predict")
async def predict(request: Request):
    started = time.time()
    data = await request.json()
    input_text = data.get("inputs", "")

    print(f"[🔔] 요청 들어옴 / 길이={len(input_text)}")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"사실관계:\n{input_text}"}
    ]

    # Qwen3 채팅 템플릿 적용
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,                # 결정론적
            temperature=0.1,                # 살짝만 랜덤성
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_ids = outputs[0][inputs.input_ids.shape[1]:]
    response_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # 마크다운 제거 및 정리
    clean_text = (
        response_text
        .replace("```json", "")
        .replace("```", "")
        .strip()
    )

    elapsed = time.time() - started
    print(f"[⏱] 처리 완료: {elapsed:.2f}초")

    return {"generated_text": clean_text}

# ==========================================
# 서버 시작 (ngrok + Threading)
# ==========================================
NGROK_TOKEN = "자신의 ngrok 토큰 입력"
ngrok.set_auth_token(NGROK_TOKEN)

# 기존 터널 정리
ngrok.kill()

# 터널 열기
public_url = ngrok.connect(8000, domain="rachele-unhappi-jin.ngrok-free.dev")

print(f"\n🚀 [API 주소]: {public_url.public_url}/predict")
print("웹 서비스 코드의 API_URL을 위 주소로 변경해서 사용하세요.\n")

# 서버 실행 함수 (별도 스레드)
def start_server():
    uvicorn.run(app, port=8000)

# 스레드로 서버 시작
thread = threading.Thread(target=start_server)
thread.start()


🚀 [API 주소]: https://rachele-unhappi-jin.ngrok-free.dev/predict
웹 서비스 코드의 API_URL을 위 주소로 변경해서 사용하세요.

